In [1]:
import tensorflow as tf

In [2]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
    print(val.numpy())

0
1
2
3
4
5
6
7
8
9


In [8]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1)
for window in dataset:
    for val in window:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


In [9]:
dataset = tf.data.Dataset.range(10)
# drop the incomplete window
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window in dataset:
    for val in window:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [19]:
def batch_window(window):
    return window.batch(5)

In [21]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(batch_window)
for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)
tf.Tensor([4], shape=(1,), dtype=int64)
[1 2 3 4 5]
tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)
tf.Tensor([5], shape=(1,), dtype=int64)
[2 3 4 5 6]
tf.Tensor([2 3 4 5], shape=(4,), dtype=int64)
tf.Tensor([6], shape=(1,), dtype=int64)
[3 4 5 6 7]
tf.Tensor([3 4 5 6], shape=(4,), dtype=int64)
tf.Tensor([7], shape=(1,), dtype=int64)
[4 5 6 7 8]
tf.Tensor([4 5 6 7], shape=(4,), dtype=int64)
tf.Tensor([8], shape=(1,), dtype=int64)
[5 6 7 8 9]
tf.Tensor([5 6 7 8], shape=(4,), dtype=int64)
tf.Tensor([9], shape=(1,), dtype=int64)


In [23]:
def window_to_x_y(window):
    return (window[:-1], window[-1:])

In [24]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(batch_window)
dataset = dataset.map(window_to_x_y)
for window, target in dataset:
    print(window.numpy(), target.numpy())
    print()

[0 1 2 3] [4]

[1 2 3 4] [5]

[2 3 4 5] [6]

[3 4 5 6] [7]

[4 5 6 7] [8]

[5 6 7 8] [9]



In [25]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(batch_window)
dataset = dataset.map(window_to_x_y)
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for window, target in dataset:
    print("window: ", window.numpy())
    print("target: ", target.numpy())

window:  [[1 2 3 4]
 [0 1 2 3]]
target:  [[5]
 [4]]
window:  [[3 4 5 6]
 [2 3 4 5]]
target:  [[7]
 [6]]
window:  [[5 6 7 8]
 [4 5 6 7]]
target:  [[9]
 [8]]


In [ ]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset